# Netflix ratings analysis

## Introduction
This project aims to analyse the ratings of Netflix movies and TV shows.

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

### Loading data

In [ ]:
try:
    df = pd.read_csv("netflix_titles.csv")
    print("Data Loaded successfully")
except FileNotFoundError:
    print("File 'netflix_titles.csv' not found")
except Exception as e:
    print("Unknown error occured: {e}")

Data Loaded successfully
